In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from sqlalchemy import create_engine
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


C:\Users\diana\AppData\Local\Temp\ipykernel_28264\325199282.py:15: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)


In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

### Q10: Produce a list of facilities with a total revenue less than 1000.The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [5]:
query = pd.read_sql_query("SELECT name, \
                                SUM(CASE WHEN memid = 0 THEN slots * guestcost ELSE slots * membercost END) AS revenue \
                                FROM Bookings \
                                LEFT JOIN Facilities \
                                USING(facid) \
                                GROUP BY name \
                                HAVING revenue < 1000;", engine)
query

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [7]:
query = pd.read_sql_query("SELECT CONCAT_WS('', a.surname, ', ', a.firstname) AS member_name, \
                            CONCAT_WS('', b.surname, ', ', b.firstname) AS recommended_by \
                            FROM Members AS a, Members AS b \
                            WHERE a.memid > 0 AND a.recommendedby = b.memid \
                            ORDER BY a.surname;", engine)
query

,member_name,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


### Q12: Find the facilities with their usage by member, but not guests

In [9]:
query = pd.read_sql_query("SELECT Facilities.name AS facility, \
                            CONCAT_WS(' ', Members.firstname, Members.surname) AS member_name, \
                            SUM(Bookings.slots) AS 'usage' \
                            FROM Bookings \
                            LEFT JOIN Facilities \
                            USING (facid) \
                            LEFT JOIN Members \
                            USING (memid) \
                            WHERE memid !=0 \
                            GROUP BY facility, member_name;", engine)
query

,facility,member_name,usage
0,Badminton Court,Anna Mackenzie,96
1,Badminton Court,Anne Baker,30
2,Badminton Court,Burton Tracy,6
3,Badminton Court,Charles Owen,18
4,Badminton Court,Darren Smith,432
...,...,...,...
197,Tennis Court 2,Ramnaresh Sarwin,36
198,Tennis Court 2,Tim Boothe,168
199,Tennis Court 2,Tim Rownam,18
200,Tennis Court 2,Timothy Baker,21


### Q13: Find the facilities usage by month, but not guests

In [11]:
query = pd.read_sql_query("SELECT strftime('%m', Bookings.starttime) AS month, \
                Facilities.name AS facility, \
                SUM(Bookings.slots ) AS 'usage' \
                FROM Bookings \
                LEFT JOIN Facilities \
                USING (facid) \
                WHERE memid !=0 \
                GROUP BY month, facility;", engine)
query

,month,facility,usage
0,07,Badminton Court,165
1,07,Massage Room 1,166
2,07,Massage Room 2,8
3,07,Pool Table,110
4,07,Snooker Table,140
5,07,Squash Court,50
6,07,Table Tennis,98
7,07,Tennis Court 1,201
8,07,Tennis Court 2,123
9,08,Badminton Court,414
